In [1]:
### Import the required libraries
import numpy as np
import scipy
import matplotlib.pyplot as plt
import cmocean

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter("ignore")

import tensorflow as tf
import tensorflow.keras as keras
from keras import metrics
from keras.constraints import NonNeg
import innvestigate
import innvestigate.utils as iutils
from innvestigate.analyzer.base import AnalyzerBase

import os
from os.path import join
import sys

import random

# See if GPUs are available
from keras import backend as K
if bool(K._get_available_gpus()):
    print("Running on GPU")
else:
    print("Running on CPU")

# Append to sys.path the absolute path to src/XAIRT
path_list = os.path.abspath('').split('/')
path_src_XAIRT = ''
for link in path_list[:-1]:
    path_src_XAIRT = path_src_XAIRT+link+'/'
sys.path.append(path_src_XAIRT+'/src')

# Now import module XAIRT
from XAIRT import *

### https://stackoverflow.com/questions/36288235/how-to-get-stable-results-with-tensorflow-setting-random-seed ###
### https://keras.io/examples/keras_recipes/reproducibility_recipes/ ###
SEED = 42
keras.utils.set_random_seed(SEED)
tf.config.experimental.enable_op_determinism()

tf.compat.v1.disable_eager_execution()

Running on CPU


2024-04-18 18:45:56.948637: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/ohpc/pub/libs/gnu12/openmpi4/netcdf/4.9.0/lib:/opt/ohpc/pub/libs/gnu12/openmpi4/netcdf-fortran/4.6.0/lib:/opt/ohpc/pub/libs/gnu12/openmpi4/hdf5/1.14.0/lib:/opt/ohpc/pub/mpi/libfabric/1.18.0/lib:/opt/ohpc/pub/mpi/ucx-ohpc/1.14.0/lib:/opt/ohpc/pub/libs/hwloc/lib:/opt/ohpc/pub/mpi/openmpi4-gnu12/4.1.5/lib:/opt/ohpc/pub/compiler/gcc/12.2.0/lib64:/share/jdk-16.0.1/lib:/home/shreyas/lis-2.1.3/installation/lib::
2024-04-18 18:45:56.948675: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2024-04-18 18:45:56.948707: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (c1-2): /proc/driver/nvidia/version does not exist
2024-04-18 18:45:56.948986: 

## Understanding LRP methods using a simple classification NN

In [2]:
N = 10000
X = np.random.rand(N,2)
y = np.dot(X,np.array([1,2]))
oneHot = np.zeros((y.shape[0],2))
oneHot[:,0] = (y >= 3)
oneHot[:,1] = (y < 3)

In [3]:
Layers = [{'size': X.shape[1], 'activation': None     , 'use_bias': None},
          {'size': 2         , 'activation': 'softmax', 'use_bias': True, 'bias_constraint': NonNeg()}]
Losses = [{'kind': 'categorical_crossentropy', 'weight': 1.0}]

NNkwargs = {'losses': Losses, 'metrics': ['mae'],
            'batch_size': 128, 'epochs': 100, 'validation_split': 0.2,
            'filename': 'model_simpleTests_LRP', 'dirname': os.path.abspath(''),
            'random_nn_seed': 42}

# learning rate schedule
def step_decay(epoch):
    initial_lrate = 0.01
    drop = 0.5
    epochs_drop = 25
    lrate = initial_lrate * drop**np.floor((1+epoch)/epochs_drop)
    return lrate

keras.backend.clear_session()
sgd = keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
NNkwargs['optimizer'] = sgd

K = TrainFullyConnectedNN(X, oneHot, layers = Layers, **NNkwargs)
best_model = K.quickTrain(step_decay)
model_wo_softmax = innvestigate.model_wo_softmax(best_model)
oneHot_NN = best_model.predict(X)

Train on 8000 samples, validate on 2000 samples
Epoch 1/100
 128/8000 [..............................] - ETA: 3s - loss: 0.4271 - mae: 0.3455
Epoch 1: val_loss improved from inf to 0.08305, saving model to /home/shreyas/XAIRT/examples_TomsQoI/model_simpleTests_LRP.h5


2024-04-18 18:45:57.689886: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled


8000/8000 [==============================] - 0s 19us/sample - loss: 0.2219 - mae: 0.1909 - val_loss: 0.0830 - val_mae: 0.0786 - lr: 0.0100
Epoch 2/100
 128/8000 [..............................] - ETA: 0s - loss: 0.0832 - mae: 0.0789
Epoch 2: val_loss improved from 0.08305 to 0.04389, saving model to /home/shreyas/XAIRT/examples_TomsQoI/model_simpleTests_LRP.h5
8000/8000 [==============================] - 0s 8us/sample - loss: 0.0625 - mae: 0.0597 - val_loss: 0.0439 - val_mae: 0.0424 - lr: 0.0100
Epoch 3/100
 128/8000 [..............................] - ETA: 0s - loss: 0.0466 - mae: 0.0451
Epoch 3: val_loss improved from 0.04389 to 0.03038, saving model to /home/shreyas/XAIRT/examples_TomsQoI/model_simpleTests_LRP.h5
8000/8000 [==============================] - 0s 8us/sample - loss: 0.0386 - mae: 0.0374 - val_loss: 0.0304 - val_mae: 0.0296 - lr: 0.0100
Epoch 4/100
 128/8000 [..............................] - ETA: 0s - loss: 0.0323 - mae: 0.0314
Epoch 4: val_loss improved from 0.03038 to 

In [4]:
idx = 2
pred_class = 1
x = X[np.newaxis,idx]
y_true = model_wo_softmax.predict(x)
W = model_wo_softmax.layers[1].get_weights()[0]
b = model_wo_softmax.layers[1].get_weights()[1]
R_last = y_true[0,pred_class]
x@W+b-y_true, y_true, W, b, x

(array([[-2.90262032e-08, -9.26665775e-08]]),
 array([[-0.57969177,  4.0242186 ]], dtype=float32),
 array([[-1.6955947,  1.2197948],
        [-2.0202467,  1.2793037]], dtype=float32),
 array([-0.       ,  3.6343434], dtype=float32),
 array([[0.15601864, 0.15599452]]))

In [5]:
np.min(X), np.max(X)

(1.1634755366141114e-05, 0.9999248268331765)

### Understanding LRP-A1B0

In [6]:
Analyze = innvestigate.create_analyzer('lrp.alpha_1_beta_0', model_wo_softmax)
a = Analyze.analyze(x)
print(f"lrp.alpha_1_beta_0: {a}, {y_true[0,pred_class]-np.sum(a)}")

lrp.alpha_1_beta_0: [[0.19031072 0.19956437]], 3.634343385696411


In [7]:
Analyze = innvestigate.create_analyzer('lrp.alpha_1_beta_0_IB', model_wo_softmax)
a = Analyze.analyze(x)
print(f"lrp.alpha_1_beta_0_IB: {a}, {R_last-np.sum(a)}")

lrp.alpha_1_beta_0_IB: [[1.9643521 2.0598667]], 0.0


In [8]:
denominator = x[0,0]*np.max(W[0,pred_class],0)+x[0,1]*np.max(W[1,pred_class],0)+2*np.max(b[pred_class],0)
R_0 = R_last * (x[0,0]*np.max(W[0,pred_class],0)+np.max(b[pred_class],0)) / denominator
R_1 = R_last * (x[0,1]*np.max(W[1,pred_class],0)+np.max(b[pred_class],0)) / denominator
print(f"Manual LRP-A1BO: {R_0},{R_1}, {R_last-R_0-R_1}")

Manual LRP-A1BO: 2.009678099610094,2.0145404596550422, 4.440892098500626e-16


In [9]:
denominator = x[0,0]*np.max(W[0,pred_class],0)+x[0,1]*np.max(W[1,pred_class],0)+np.max(b[pred_class],0)
R_0 = R_last * (x[0,0]*np.max(W[0,pred_class],0)) / denominator
R_1 = R_last * (x[0,1]*np.max(W[1,pred_class],0)) / denominator
print(f"Manual LRP-A1BO without bias in numerator and only one bias in denominator: {R_0},{R_1}, {R_last-R_0-R_1}")

Manual LRP-A1BO without bias in numerator and only one bias in denominator: 0.19031072292947834,0.1995643669504099, 3.6343434693852483


In [10]:
denominator = x[0,0]*np.max(W[0,pred_class],0)+x[0,1]*np.max(W[1,pred_class],0)
R_0 = R_last * (x[0,0]*np.max(W[0,pred_class],0)) / denominator
R_1 = R_last * (x[0,1]*np.max(W[1,pred_class],0)) / denominator
print(f"Manual LRP-A1BO without bias in both numerator and denominator: {R_0},{R_1}, {R_last-R_0-R_1}")

Manual LRP-A1BO without bias in both numerator and denominator: 1.9643520787027295,2.059866480562407, 0.0


### Understanding LRP-W2

In [11]:
Analyze = innvestigate.create_analyzer('lrp.alpha_1_beta_0', model_wo_softmax, input_layer_rule='WSquare')
a = Analyze.analyze(x)
print(f"lrp.alpha_1_beta_0: {a}, {R_last-np.sum(a)}")

lrp.alpha_1_beta_0: [[1.9163382 2.1078804]], 0.0


In [12]:
Analyze = innvestigate.create_analyzer('lrp.alpha_1_beta_0_IB', model_wo_softmax, input_layer_rule='WSquare')

a = Analyze.analyze(x)

print(f"lrp.alpha_1_beta_0_IB: {a}, {R_last-np.sum(a)}")

lrp.alpha_1_beta_0_IB: [[1.9163382 2.1078804]], 0.0


In [13]:
denominator = W[0,pred_class]**2 + W[1,pred_class]**2
R_0 = R_last * W[0,pred_class]**2 / denominator
R_1 = R_last * W[1,pred_class]**2 / denominator
print(f"Manual LRP-WSquare: {R_0},{R_1}, {R_last-R_0-R_1}")

Manual LRP-WSquare: 1.9163382662566082,2.1078802930085283, 0.0


### Understanding LRP-Bounded

In [14]:
Analyze = innvestigate.create_analyzer('lrp.alpha_1_beta_0', model_wo_softmax, input_layer_rule='Bounded')
a = Analyze.analyze(x)
print(f"lrp.alpha_1_beta_0 bounded: {a}, {R_last-np.sum(a)}")

lrp.alpha_1_beta_0 bounded: [[1.9642177 2.060001 ]], 0.0


In [15]:
Analyze = innvestigate.create_analyzer('lrp.alpha_1_beta_0', model_wo_softmax, input_layer_rule=(13,9))
a = Analyze.analyze(x)
print(f"lrp.alpha_1_beta_0 custom bounds: {a}, {R_last-np.sum(a)}")

lrp.alpha_1_beta_0 custom bounds: [[1.9641943 2.0600235]], 9.5367431640625e-07


In [16]:
Analyze = innvestigate.create_analyzer('lrp.alpha_1_beta_0_IB', model_wo_softmax, input_layer_rule='Bounded')
a = Analyze.analyze(x)
print(f"lrp.alpha_1_beta_0_IB bounded: {a}, {R_last-np.sum(a)}")

lrp.alpha_1_beta_0_IB bounded: [[1.9642177 2.060001 ]], 0.0


In [17]:
Analyze = innvestigate.create_analyzer('lrp.alpha_1_beta_0_IB', model_wo_softmax, input_layer_rule=(13,9))
a = Analyze.analyze(x)
print(f"lrp.alpha_1_beta_0_IB custom bounds: {a}, {R_last-np.sum(a)}")

lrp.alpha_1_beta_0_IB custom bounds: [[1.9641943 2.0600235]], 9.5367431640625e-07


In [18]:
low = -1
high = 1
denominator = x[0,0]*W[0,pred_class]+x[0,1]*W[1,pred_class] \
            -low*(np.max(W[0,pred_class],0)+np.max(W[1,pred_class],0)) \
            -high*(np.min(W[0,pred_class],0)+np.min(W[1,pred_class],0))
R_0 = R_last * (x[0,0]*W[0,pred_class]-low*np.max(W[0,pred_class],0)-high*np.min(W[0,pred_class],0)) / denominator
R_1 = R_last * (x[0,1]*W[1,pred_class]-low*np.max(W[1,pred_class],0)-high*np.min(W[1,pred_class],0)) / denominator
print(f"Manual LRP-Bounded: {R_0},{R_1}, {R_last-R_0-R_1}")

Manual LRP-Bounded: 1.9643520787027293,2.0598664805624076, -4.440892098500626e-16


In [19]:
low = -20
high = 20
denominator = x[0,0]*W[0,pred_class]+x[0,1]*W[1,pred_class] \
            -low*(np.max(W[0,pred_class],0)+np.max(W[1,pred_class],0)) \
            -high*(np.min(W[0,pred_class],0)+np.min(W[1,pred_class],0))
R_0 = R_last * (x[0,0]*W[0,pred_class]-low*np.max(W[0,pred_class],0)-high*np.min(W[0,pred_class],0)) / denominator
R_1 = R_last * (x[0,1]*W[1,pred_class]-low*np.max(W[1,pred_class],0)-high*np.min(W[1,pred_class],0)) / denominator
print(f"Manual LRP-Bounded with custom bounds: {R_0},{R_1}, {R_last-R_0-R_1}")

Manual LRP-Bounded with custom bounds: 1.9643520787027318,2.0598664805624045, 4.440892098500626e-16


In [20]:
low = 0.4
high = 0.6
denominator = x[0,0]*W[0,pred_class]+x[0,1]*W[1,pred_class] \
            -low*(np.max(W[0,pred_class],0)+np.max(W[1,pred_class],0)) \
            -high*(np.min(W[0,pred_class],0)+np.min(W[1,pred_class],0))
R_0 = R_last * (x[0,0]*W[0,pred_class]-low*np.max(W[0,pred_class],0)-high*np.min(W[0,pred_class],0)) / denominator
R_1 = R_last * (x[0,1]*W[1,pred_class]-low*np.max(W[1,pred_class],0)-high*np.min(W[1,pred_class],0)) / denominator
print(f"Manual LRP-Bounded with custom bounds: {R_0},{R_1}, {R_last-R_0-R_1}")

Manual LRP-Bounded with custom bounds: 1.9641679967078447,2.0600507899984803, -2.2744118854944873e-07


In [21]:
N = 1000000
X = np.random.rand(N,2)
y = np.dot(X,np.array([1,2]))[:,np.newaxis]
oneHot = np.zeros([y.shape[0],2])
oneHot[:,0] = ((y[:,0] >= 1.) & (y[:,0] <= 2.))
oneHot[:,1] = ((y[:,0] < 1.) | (y[:,0] > 2.))

np.sum(oneHot[:,0]), np.sum(oneHot[:,1])

(500060.0, 499940.0)

In [ ]:
Layers = [{'size': X.shape[1], 'activation': None     , 'use_bias': None},
          {'size': 8         , 'activation': 'relu'   , 'use_bias': True},
          {'size': 8         , 'activation': 'relu'   , 'use_bias': True},
          {'size': 2         , 'activation': 'softmax', 'use_bias': True}]
Losses = [{'kind': 'categorical_crossentropy', 'weight': 1.0}]

NNkwargs = {'losses': Losses, 'metrics': ['mae'],
            'batch_size': 2048, 'epochs': 100, 'validation_split': 0.2,
            'filename': 'model_simpleTests_OI', 'dirname': os.path.abspath(''),
            'random_nn_seed': 42}

# learning rate schedule
def step_decay(epoch):
    initial_lrate = 0.01
    drop = 0.5
    epochs_drop = 25
    lrate = initial_lrate * drop**np.floor((1+epoch)/epochs_drop)
    return lrate

keras.backend.clear_session()
sgd = keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
NNkwargs['optimizer'] = sgd

K = TrainFullyConnectedNN(X, oneHot, layers = Layers, **NNkwargs)
best_model = K.quickTrain(step_decay)
oneHot_NN = best_model.predict(X)

Train on 800000 samples, validate on 200000 samples
Epoch 1/100
770048/800000 [===========================>..] - ETA: 0s - loss: 0.5718 - mae: 0.4180
Epoch 1: val_loss improved from inf to 0.27812, saving model to /home/shreyas/XAIRT/examples_TomsQoI/model_simpleTests_OI.h5
800000/800000 [==============================] - 1s 1us/sample - loss: 0.5611 - mae: 0.4112 - val_loss: 0.2781 - val_mae: 0.2297 - lr: 0.0100
Epoch 2/100
761856/800000 [===========================>..] - ETA: 0s - loss: 0.1655 - mae: 0.1389
Epoch 2: val_loss improved from 0.27812 to 0.10768, saving model to /home/shreyas/XAIRT/examples_TomsQoI/model_simpleTests_OI.h5
800000/800000 [==============================] - 1s 1us/sample - loss: 0.1628 - mae: 0.1366 - val_loss: 0.1077 - val_mae: 0.0908 - lr: 0.0100
Epoch 3/100
745472/800000 [==========================>...] - ETA: 0s - loss: 0.0879 - mae: 0.0742
Epoch 3: val_loss improved from 0.10768 to 0.07351, saving model to /home/shreyas/XAIRT/examples_TomsQoI/model_simpl

Class 1: $x_1 + 2x_2 \in [1,2]$, Class 2 is the opposite.

In [ ]:
inp_numpy = np.array([[0.,0.]])
desired_labels_numpy = np.array([[1.,0.]])
desired_labels = tf.convert_to_tensor(desired_labels_numpy)

print(f"Desired label : {desired_labels_numpy}")
for i in range(100):
    inp = tf.convert_to_tensor(inp_numpy)
    grads = GradientDescent_useGradientTape(best_model, inp, desired_labels, 
                            keras.losses.BinaryCrossentropy())
    grads_numpy = np.squeeze(tf_to_numpy(grads))
    inp_numpy[0,:] = inp_numpy[0,:] - 0.01*grads_numpy
    if (i+1)%10 == 0:
        print(f"Iter {i+1}, Prediction {tf_to_numpy(best_model.predict(inp_numpy))}")
    
print(f"Optimal input is : {inp_numpy}")

inp_numpy = np.array([[0.3,0.6]]) # Optimal input for opposite class to start off
desired_labels_numpy = np.array([[0.,1.]])
desired_labels = tf.convert_to_tensor(desired_labels_numpy)

print(f"Desired label : {desired_labels_numpy}")
for i in range(100):
    inp = tf.convert_to_tensor(inp_numpy)
    grads = GradientDescent_useGradientTape(best_model, inp, desired_labels, 
                            keras.losses.BinaryCrossentropy())
    grads_numpy = np.squeeze(tf_to_numpy(grads))
    inp_numpy[0,:] = inp_numpy[0,:] - 0.01*grads_numpy
    if (i+1)%10 == 0:
        print(f"Iter {i+1}, Prediction {tf_to_numpy(best_model.predict(inp_numpy))}")
    
print(f"Optimal input is : {inp_numpy}")